In [1]:
!pip install Levenshtein

     |████████████████████████████████| 163kB 5.2MB/s 


In [2]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from collections import OrderedDict
from time import time
from tqdm import tqdm

import numpy as np
import scipy as sp
from scipy.sparse import csr_matrix
import pandas as pd

from scipy.optimize import fmin_powell
from scipy import integrate
from scipy import linalg

from sklearn.preprocessing import normalize, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn import linear_model
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

np.set_printoptions(precision=4, suppress=True)

from collections import Counter
from Levenshtein import distance as levenshtein_distance

sns.set_style("whitegrid")
sns.set_palette("colorblind")
palette = sns.color_palette()
figsize = (15,8)
legend_fontsize = 16

from matplotlib import rc
rc('font',**{'family':'sans-serif'})
rc('figure', **{'dpi' : 200})

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Спортивное "Что? Где? Когда?": данные

In [4]:
import json, pickle

tournaments = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/Advanced_ML/2HW/tournaments.pkl', 'rb'))
results = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/Advanced_ML/2HW/results.pkl', 'rb'))
players = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/Advanced_ML/2HW/players.pkl', 'rb'))

# 1.

Выберем данные, в которых есть данные о составах команд и также заметим, что есть турниры, в которых количество вопросов у разных команд разное. В таких турнирах оставим только те команды, которые приняли участие во всех вопросах, а другие команды будем считать дисквалифицированными и не будем их учитывать.

In [5]:
for res in results[5864]:
    print('Results:', res['mask'])

Results: 111111111001110111101110111110111111111111101111110110010000000111111100110011101111000101001001111101101001001010010101100001
Results: 111011110001101011101110011100101011101111101001100111001101110111010100111111100111001011000101110101111000011011010101100101
Results: 101011100001101011111110011000101111101111101001010110110011100111010001111010111111001100100010110101101101101010010101111101
Results: 100111111001101011001000111001101011101111101001100110110000000010110000111111100011000000100001001101001000001110011101110111
Results: 110011101001110011101010011000101011101111001011100101010000000111110000111000000011110010100110110101110010001010010100110111
Results: 101111101000111110110010100111010100111110010111100111101111100101111110101110011101110111
Results: 110111101111011110110001110111111000111011110111001001100101111101011101011110001001110101
Results: 111011110001100011001110001000101101100111101000010111010000000111010110101011000011010001110001100001111100001

In [6]:
train = [ (k, v['name']) for k,v in tournaments.items() if v['dateStart'][:4] == '2019' ]
test = [ (k, v['name']) for k,v in tournaments.items() if v['dateStart'][:4] == '2020' ]

In [7]:
train_new = []
test_new = []
tournaments_2019_2020 = set()
players_2019_2020 = set()
num_questions_train = 0

for id_, name in train:
    if not results[id_] or 'mask' not in results[id_][0].keys() or not results[id_][0]['mask']:
        continue
    leng = len(results[id_][0]['mask'])
    is_correct = True
    for res in results[id_]:
        if not res['mask'] or len(res['mask']) != leng:
            is_correct = False
            break
    if is_correct:
        teams = []
        for res in results[id_]:
            if res['teamMembers']:
                if not res['mask'].isdigit():
                    mask = ''
                    for x in res['mask']:
                        if x.isdigit():
                            mask += x
                    score_list = list(map(int, list(mask)))
                else:
                    score_list = list(map(int, list(res['mask'])))
                score = sum(score_list) / len(score_list)
                teams.append(([p['player']['id'] for p in res['teamMembers']], score_list))
                num_questions_train += len(score_list)
                for p in res['teamMembers']:
                    players_2019_2020.add(p['player']['id'])
            else:
                continue
        train_new.append((id_, teams))
    tournaments_2019_2020.add(id_)
        

for id_, name in test:
    if not results[id_] or 'mask' not in results[id_][0].keys() or not results[id_][0]['mask']:
        continue
    leng = len(results[id_][0]['mask'])
    is_correct = True
    for res in results[id_]:
        if not res['mask'] or len(res['mask']) != leng:
            is_correct = False
            break
    if is_correct:
        teams = []
        for res in results[id_]:
            if res['teamMembers']:
                if not res['mask'].isdigit():
                    mask = ''
                    for x in res['mask']:
                        if x.isdigit():
                            mask += x
                    score_list = list(map(int, list(mask)))
                else:
                    score_list = list(map(int, list(res['mask'])))
                score = sum(score_list) / len(score_list)
                teams.append(([p['player']['id'] for p in res['teamMembers']], score_list))
                for p in res['teamMembers']:
                    players_2019_2020.add(p['player']['id'])
            else:
                continue
        test_new.append((id_, teams))
    tournaments_2019_2020.add(id_)
    
tournaments_2019_2020 = list(tournaments_2019_2020)
players_2019_2020 = list(players_2019_2020)

In [8]:
train_new[0]

(4772,
 [([6212, 18332, 18036, 22799, 15456, 26089],
   [1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    0,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    0,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    0,
    0,
    0,
    1,
    0,
    0,
    1,
    0]),
  ([1585, 40840, 1584, 10998, 16206],
   [1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    0,
    1,
    1,
    1,
    1,
    0,
    1,
    0,
    0,
    1,
    0,
    1,
    1,
    1,
    1,
    0,
    1,
    1,
    0,
    0,
    1,
    0,
    1,
    1,
    0,
    1,
    0]),
  ([23513, 18168, 21060, 35850, 31332, 10187],
   [1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    1,
    0,
    1,
    1,
    1,
    1,
    0,
    1,
    0,
    1,
    1,
    0,
    1,
    1,
    1,
    1,
    0,
    0,
    1,
    0,
    1,
    1,
    1,
    1,
    0,
    0,
    0,
    0]),
  ([36742, 28939, 54289, 15381, 27375],
   [1,
    0,
    1,
    1,
    1,
    1,
    1,
    0,
    

# 2 - 3.

__Способ 1__ (обучать эффективность каждого игрока):

Для построения модели для ранжирования игроков будем использовать информацию о том, ответил ли игрок на вопрос или не ответил для каждого вопроса. Построим логистическую регрессию на тренировочных данных и сравним веса, с которыми учитываются разные игроки и разные вопросы.

Модель можно интерпретировать следующим образом - мы обучаем скилл игрока и сложность вопроса лог регрессией:

$ p(x_{ij}) = \sigma(X * w)_{f(i, j)} = \sigma(skill_i + difficulty_j) $,

где $x_{ij}$ - факт ответа на j вопрос i-м игроком, $f(i, j) $ - функция, которая возвращяет индекс строки матрицы X признаков с 1 на i-ом и j-ом местах и остальными нулями

In [9]:
from sklearn.linear_model import LinearRegression, Ridge, LogisticRegression

In [10]:
X_train = []
y_train = []

n_questions = 0
for tour in tqdm(train_new):
    for team in tour[1]:
        for player in team[0]:
            for j in range(len(team[1])):
                X_train.append(np.array([player, n_questions + j]))
                y_train.append(team[1][j])
    n_questions += len(tour[1][0][1])

X_train = np.array(X_train)

100%|██████████| 660/660 [00:32<00:00, 20.15it/s]


In [11]:
ohe = OneHotEncoder(sparse=True).fit(X_train)
X_train = ohe.transform(X_train)
y_train = np.array(y_train)

In [12]:
X_train.shape, y_train.shape

((16119246, 87498), (16119246,))

In [13]:
%%time

log_reg = LogisticRegression().fit(X_train, y_train)

CPU times: user 3min 37s, sys: 25.6 s, total: 4min 2s
Wall time: 3min 34s


In [14]:
log_reg.coef_

array([[-0.0358,  0.8907,  0.1102, ...,  1.869 , -0.7312, -0.2922]])

In [15]:
ohe.categories_

[array([    15,     16,     23, ..., 224482, 224539, 224542]),
 array([    0,     1,     2, ..., 30630, 30631, 30632])]

In [16]:
players_skill = MinMaxScaler().fit_transform(log_reg.coef_[0][:ohe.categories_[0].shape[0]].reshape(ohe.categories_[0].shape[0], 1))
question_difficulty = MinMaxScaler().fit_transform(log_reg.coef_[0][ohe.categories_[0].shape[0]:].reshape(ohe.categories_[1].shape[0], 1))

players_skill = np.concatenate((ohe.categories_[0].reshape(ohe.categories_[0].shape[0], 1), players_skill), axis=1)
question_difficulty = np.concatenate((ohe.categories_[1].reshape(ohe.categories_[1].shape[0], 1), question_difficulty), axis=1)

In [17]:
players_skill_dict = dict(players_skill)
question_difficulty_dict = dict(question_difficulty)

In [18]:
print('\n'.join([' '.join([str(i + 1), ' ', players[int(k)]['surname'], players[int(k)]['name'], players[int(k)]['patronymic'], '\t\t', str(v)]) \
                         for i, (k, v) in enumerate(sorted(players_skill, key=lambda x: x[1], reverse=True)[:20])]))

0   Брутер Александра Владимировна 		 1.0
1   Руссо Максим Михайлович 		 0.99586151804462
2   Семушин Иван Николаевич 		 0.9807091089514975
3   Савченков Михаил Владимирович 		 0.9755352709147873
4   Сорожкин Артём Сергеевич 		 0.9715619324964967
5   Юнгер Мария Алексеевна 		 0.9437971608563993
6   Спешков Сергей Леонидович 		 0.9399567687667126
7   Бровер Сусанна Марковна 		 0.9302169650450905
8   Михайлов Кирилл Игоревич 		 0.9291191632634037
9   Митрофанов Артём Александрович 		 0.9206692374371003
10   Дубинская Ольга Игоревна 		 0.9145675125605137
11   Богословский Алексей Алексеевич 		 0.9143100136447077
12   Саксонов Антон Владимирович 		 0.9112923452634092
13   Царёв Михаил Сергеевич 		 0.9093923826377516
14   Иванцова Светлана Сергеевна 		 0.9055411558136148
15   Мокин Игорь Викторович 		 0.9034978798488931
16   Марков Александр Валерьевич 		 0.9030463701320759
17   Кудряшова Наталья Алексеевна 		 0.9016364774391163
18   Бомешко Светлана Борисовна 		 0.9002212247736538
19   Ник

In [19]:
def find_scores(players_skill_dict):
    spearman_scores = []
    kendall_scores = []


    for tour in test_new:
        real_ranks = []
        pred_ranks = []
        for i, team in enumerate(tour[1]):
            real_ranks.append((i, sum(team[1]) / len(team[1])))
            score = 0
            k = 0
            for p in team[0]:
                if p in players_skill_dict.keys():
                    score += players_skill_dict[p]
                    k += 1
            if k:
                pred_ranks.append((i, score / k))
            else:
                pred_ranks.append((i, 0))

        real_ranks = [k for k, v in sorted(real_ranks, key=lambda x: x[1], reverse=True)]
        pred_ranks = [k for k, v in sorted(pred_ranks, key=lambda x: x[1], reverse=True)]

        spearman_sc = sp.stats.spearmanr(real_ranks, pred_ranks)[0]
        kendall_sc = sp.stats.kendalltau(real_ranks, pred_ranks)[0]
        if not np.isnan(spearman_sc):
            spearman_scores.append(spearman_sc)
        if not np.isnan(kendall_sc):
            kendall_scores.append(kendall_sc)

    print(f'Spearman average score is {sum(spearman_scores)/len(spearman_scores)}')
    print(f'Kendall average score is {sum(kendall_scores)/len(kendall_scores)}')

In [20]:
find_scores(players_skill_dict)

Spearman average score is 0.7593225630823455
Kendall average score is 0.5925790501031619


Получили достаточно адекватыне результаты и список топ игроков

__Способ 2__ (непосредственно обучать эффективность всей команды от ее состава):

Теперь будем сразу строить модель для ранжирования команд с известными составами. Соберем датасет следующий образом: объектом будет служить команда, а признаками - вектора длины всех игроков с 1 на месте игравших в данной команде игроков и с остальными нулями. Таргетом будет эффективность команды в игре (считаем, что эффективность команды совпадает с эффективностью игроков, входящих в нее).

Размерность таких матриц будет огромной и уместить их в np.array не представляется возможным, зато можно без труда уместить их в sparse matrix представление. И, так как у нас матрица состоит только из 0 и 1, то будем хранить данные в int8. На этом будем обучать линейную регрессию.

In [21]:
train_len2 = 0
train_row2 = []
train_col2 = []
train_data2 = []

y_train2 = []

i = 0
for tour in tqdm(train_new):
    for team in tour[1]:
        for player in team[0]:
            train_row2.append(i)
            train_col2.append(player)
            train_data2.append(1)
        y_train2.append(sum(team[1]) / len(team[1]))
        train_len2 += 1
        i += 1

train_row2 = np.array(train_row2)
train_col2 = np.array(train_col2)
train_data2 = np.array(train_data2, dtype=np.int8)

100%|██████████| 660/660 [00:00<00:00, 1977.83it/s]


In [22]:
test_len2 = 0
test_row2 = []
test_col2 = []
test_data2 = []

i = 0
for tour in tqdm(test_new):
    for team in tour[1]:
        for player in team[0]:
            test_row2.append(i)
            test_col2.append(player)
            test_data2.append(1)
        test_len2 += 1
        i += 1

test_row2 = np.array(test_row2)
test_col2 = np.array(test_col2)
test_data2 = np.array(test_data2, dtype=np.int8)

100%|██████████| 165/165 [00:00<00:00, 3126.26it/s]


In [23]:
X_train2 = csr_matrix((train_data2, (train_row2, train_col2)), shape=(train_len2, max(players.keys())), dtype=np.int8)
y_train2 = np.array(y_train2)
X_test2 = csr_matrix((test_data2, (test_row2, test_col2)), shape=(test_len2, max(players.keys())), dtype=np.int8)

In [24]:
%%time

#linear_reg = LinearRegression().fit(X_train, y_train)
linear_reg = Ridge(alpha=20).fit(X_train2, y_train2)

CPU times: user 351 ms, sys: 336 ms, total: 687 ms
Wall time: 381 ms


Теперь предскажем эффективность каждой команды в тесте и отранжируем команды по реальным данным и по предсказанным. Воспользуемся ранговыми корреляциями Спирмена и Кендалла для получения метрики качества.

In [25]:
y_pred = linear_reg.predict(X_test2)

In [26]:
spearman_scores = []
kendall_scores = []

j = 0
for tour in test_new:
    real_ranks = []
    pred_ranks = []
    for i, team in enumerate(tour[1]):
        real_ranks.append((i, sum(team[1]) / len(team[1])))
        pred_ranks.append((i, y_pred[j]))
        j += 1
    real_ranks = [k for k, v in sorted(real_ranks, key=lambda x: x[1], reverse=True)]
    pred_ranks = [k for k, v in sorted(pred_ranks, key=lambda x: x[1], reverse=True)]
    
    spearman_sc = sp.stats.spearmanr(real_ranks, pred_ranks)[0]
    kendall_sc = sp.stats.kendalltau(real_ranks, pred_ranks)[0]
    if not np.isnan(spearman_sc):
        spearman_scores.append(spearman_sc)
    if not np.isnan(kendall_sc):
        kendall_scores.append(kendall_sc)

In [27]:
print(f'Spearman average score is {sum(spearman_scores)/len(spearman_scores)}')
print(f'Kendall average score is {sum(kendall_scores)/len(kendall_scores)}')

Spearman average score is 0.7121346799255368
Kendall average score is 0.544186349525721


Первый способ имеет более высокие показатели, а также позволяет сравнивать не только команды, но и игроков, поэтому далее будем опираться именно на подход с логистической регрессией.

# 4.

Добавим скрытую переменную $z_{ij}$ - игрок i правильно ответил на вопрос j. Теперь, если мы знаем эту переменную, то без труда можем ранжировать команды на турнирах. Заметим, что если команда правильно ответила на вопрос, то хотя бы для одного игрока этой команды $z = 1$, а если команда не ответила на вопрос, то $z = 0$ для всех игроков команды. То есть теперь мы хотим показать, что заслуга ответа на вопрос лежит не на одном игроке, а распределяется по членам команды с какими-то коэффициентами. 

Обучать будем те же параметры, только теперь при помощи ЕМ-алгоритма

$ p(z_{ij}) = \sigma(X * w)_{f(i, j)} = \sigma(skill_i + difficulty_j) $

* __Е-step__:

Фиксируем $skill_i$ и $difficulty_j$ и вычисляем скрытые переменные

$\mathop{\mathbb{E}} [z_{ij}] = \begin{cases}
0, & x_{ij} = 0 \\ 
\frac{\sigma(skill_i + difficulty_j)}{p_{\text{team response}}} =  
\frac{\sigma(skill_i + difficulty_j)}{1 - \prod_k(1 - \sigma(skill_k + difficulty_j))}, & x_{ij} = 1
\end{cases} $ 


* __M-step__:

Фиксируем $z_{ij}$ и пересчитываем $skill_i$ и $difficulty_j$

$$\mathop{\mathbb{E}} [z_{ij}] = \sigma(skill_i + difficulty_j) $$
$$ X * (skill_i, difficulty_j) = \sigma^{-1}(\mathop{\mathbb{E}} [z_{ij}]) $$

Обучая линейную регрессию, выражаем искомые веса

$$ (skill_i, difficulty_j) = (X^TX)^{-1}X^T \sigma^{-1}(\mathop{\mathbb{E}} [z_{ij}]) $$


\\
Проделаем несколько шагов и посмотрим на результаты

In [31]:
def sigmoid(skill, difficulty):
    return 1 / (1 + np.exp(-(skill + difficulty)))


def e_step(w):
    print('E-step running...')
    y = X_train @ w
    y = 1 / (1 + np.exp(-y))

    labels = []
    n_questions = 0
    i = 0
    for tour in train_new:
        n_questions += len(tour[1][0][1])
        for team in tour[1]:
            fix = i
            num_q = len(team[1])
            n_players = len(team[0])
            for l in range(n_players):
                for j in range(num_q):
                    if y_train[i] == 0:
                        labels.append(1e-4)
                        i += 1
                        continue
                    p_not_resp = 1
                    for k in range(n_players):
                        p_not_resp *= (1 - y[fix + num_q * k + j])
                    p_resp = 1 - p_not_resp
                    p_resp_by_player = y[i] / p_resp
                    if p_resp_by_player < 1e-4:
                        labels.append(1e-4)
                    elif p_resp_by_player > 1 - 1e-4:
                        labels.append(1 - 1e-4)
                    else:
                        labels.append(p_resp_by_player)
                    i += 1
    return np.array(labels).astype(np.float32)


def m_step(labels):
    print('M-step running...')
    y = - np.log(1 / labels - 1)
    lin_reg = LinearRegression().fit(X_train, y)
    return lin_reg.coef_[:ohe.categories_[0].shape[0]], lin_reg.coef_[ohe.categories_[0].shape[0]:]


def check_metrics(skill_weights, difficulty_weights):
    players_skill = MinMaxScaler().fit_transform(skill_weights.reshape(skill_weights.shape[0], 1))
    question_difficulty = MinMaxScaler().fit_transform(difficulty_weights.reshape(difficulty_weights.shape[0], 1))

    players_skill = np.concatenate((ohe.categories_[0].reshape(ohe.categories_[0].shape[0], 1), players_skill), axis=1)
    question_difficulty = np.concatenate((ohe.categories_[1].reshape(ohe.categories_[1].shape[0], 1), question_difficulty), axis=1)
    
    players_skill_dict = dict(players_skill)
    question_difficulty_dict = dict(question_difficulty)
    
    return players_skill_dict, question_difficulty_dict

In [32]:
e_labels = np.zeros(X_train.shape[1])

NUM_ITERS = 5
for i in range(NUM_ITERS):
    print(f'ITERATION {i + 1}')
    m_labels = e_step(e_labels)
    skill_weights, difficulty_weights = m_step(m_labels)
    players_skill_dict, question_difficulty_dict = check_metrics(skill_weights, difficulty_weights)
    find_scores(players_skill_dict)
    e_labels = np.concatenate((skill_weights, difficulty_weights)).reshape(skill_weights.shape[0] + difficulty_weights.shape[0], 1)
    print('\n')

ITERATION 1
E-step running...
M-step running...
Spearman average score is 0.779876896223343
Kendall average score is 0.61457860546081


ITERATION 2
E-step running...
M-step running...
Spearman average score is 0.7796432575644021
Kendall average score is 0.6141478287458371


ITERATION 3
E-step running...
M-step running...
Spearman average score is 0.7818775824016162
Kendall average score is 0.6172996168443919


ITERATION 4
E-step running...
M-step running...
Spearman average score is 0.7791989478128362
Kendall average score is 0.613343263498367


ITERATION 5
E-step running...
M-step running...
Spearman average score is 0.7791373205012777
Kendall average score is 0.613401670309775




ЕМ-алгоритм работает - видим неплохие результаты уже с 1-ой итерации, далее улучшение незначительно. При этом получилось получить результаты выше, чем у baseline моделей

Посмотрим на новый рейтинг игроков:

In [36]:
print('\n'.join([' '.join([str(i + 1), ' ', players[int(k)]['surname'], players[int(k)]['name'], players[int(k)]['patronymic'], '\t\t', str(v)]) \
                         for i, (k, v) in enumerate(sorted(players_skill_dict.items(), key=lambda x: x[1], reverse=True)[:20])]))

1   Руссо Максим Михайлович 		 1.0
2   Пилипенко Максим Игоревич 		 0.9940174240034658
3   Брутер Александра Владимировна 		 0.9828114873340144
4   Калинина Анастасия Борисовна 		 0.9815661831759654
5   Семушин Иван Николаевич 		 0.9808052427184137
6   Кудинов Дмитрий Сергеевич 		 0.9728985571996136
7   Савченков Михаил Владимирович 		 0.9709341822468343
8   Сорожкин Артём Сергеевич 		 0.9577424403456023
9   Спешков Сергей Леонидович 		 0.9547437990870373
10   Мереминский Станислав Григорьевич 		 0.9465186300292407
11   Левандовский Михаил Ильич 		 0.932672346120171
12   Спектор Евгений Павлович 		 0.9292688344351568
13   Прокофьева Ирина Сергеевна 		 0.9289473788660352
14   Мосягин Александр Владимирович 		 0.9280243546043212
15   Новиков Илья Сергеевич 		 0.9251997729216996
16   Плохих Владислава Вадимовна 		 0.9248714916982497
17   Завьялов Сергей Андреевич 		 0.9247928390957851
18   Бровер Сусанна Марковна 		 0.9230312883532008
19   Гилёв Алексей Владимирович 		 0.9223854385174055


# 5.

Теперь попробуем по полученным результатам построить рейтинг-лист турниров по сложности.

In [44]:
tournament_rating = dict()

n_questions = 0
for tour in train_new:
    num_q = len(tour[1][0][1])
    for i in range(n_questions, n_questions + num_q):
        tournament_rating[tour[0]] = question_difficulty_dict[i]
    n_questions += num_q

In [51]:
print('\n'.join([' '.join([str(i + 1), ' ', tournaments[k]['name'], '\t\t', str(v)]) \
                         for i, (k, v) in enumerate(sorted(tournament_rating.items(), key=lambda x: x[1], reverse=False)[:20])]))

1   Чемпионат Мира. Этап 2. Группа А 		 0.08327803488121244
2   Мемориал Дмитрия Коноваленко 		 0.09990696379904684
3   Знание – Сила VI 		 0.11014432463003054
4   Чемпионат России 		 0.11226936501705642
5   Чемпионат МГУ. Высшая лига. Четвёртый игровой день 		 0.11938801626102596
6   Кубок Москвы 		 0.1255282032719472
7   Синхрон Беловежской Зимы. День второй. 		 0.12613770525182672
8   Чемпионат Мира. Этап 2. Группа В 		 0.1290709864449297
9   Чемпионат Санкт-Петербурга. Первая лига 		 0.1320491054687335
10   Игра Первопрестольной. Четвёртый сезон 		 0.13427787962058185
11   Синхронный Кубок Хайфы 		 0.1382461633995618
12   Львов зимой. Барон 		 0.13841239065259497
13   Антибинго 		 0.14131581689563927
14   Кубок Кайнозоя 		 0.1429891165156124
15   Линч 		 0.14402999866778476
16   Открытый Кубок Зеленограда 		 0.1458948417285218
17   Ревельская весна 		 0.1473054869682282
18   Зеркало Симбирск-Open 		 0.14760802219315894
19   Синхрон Беловежской Зимы. День первый. 		 0.14911076653024

Список практически соответствует интуиции, но теперь видим, что мы на самом деле оценивали не сложность вопроса, а наоборот его легкость.